<a href="https://colab.research.google.com/github/ShawnLiu119/Segmentation_Embedding_DL/blob/main/Prototyping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer2Vec Models - Prototype**

In this prototype, we focus on a basic scenario where customer event histories are generated using a simple Markov chain. We demonstrate that Word2vec and Doc2vec models capture sequential event patterns.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
import random

from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.decomposition import TruncatedSVD

## Step1: generate fake data

We consider a toy model of an online apparel store that sells two categories of products, hats and dresses. Each category is represented by two products, so we have 4 products in total, 2 hats and 2 dresses. We assume two classes of customers:

-  Category shoppers come to buy either a hat or a dress. They tend to browse within one category but rarely switch between categories.
-  Look shoppers come to buy both a hat and a dress. They tend to frequently jump between the categories.<br>
Each of these two classes of shoppers is specified using a stochastic transition matrix.

In [4]:
#
# Shopper transition matrices
#

# Category shopper
transition_matrix_a = np.array([
# Hat 1 | Hat 2 | Dress 1 | Dress 2
  [0.1,    0.9,    0.0,      0.0],      # Hat 1
  [0.8,    0.1,    0.1,      0.0],      # Hat 2
  [0.0,    0.0,    0.1,      0.9],      # Dress 1
  [0.1,    0.0,    0.8,      0.1]       # Dress 2
])

# Look shoper
transition_matrix_b = np.array([
# Hat 1 | Hat 2 | Dress 1 | Dress 2
  [0.1,    0.9,    0.0,      0.0],      # Hat 1
  [0.4,    0.1,    0.5,      0.0],      # Hat 2
  [0.0,    0.0,    0.1,      0.9],      # Dress 1
  [0.5,    0.0,    0.4,      0.1]       # Dress 2
])

#
# Stationary distribution over products
#
def stationary_distribution(transition_matrix):
    P = np.transpose(transition_matrix) - np.identity(transition_matrix.shape[0])
    A = np.append(P, np.ones((1, P.shape[0])), axis=0)
    b = np.transpose(np.append(np.zeros(P.shape[0]), [1]))
    return np.linalg.solve(np.transpose(A).dot(A), np.transpose(A).dot(b))

print(stationary_distribution(transition_matrix_a))
print(stationary_distribution(transition_matrix_b))

[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]


In [5]:
np.transpose(transition_matrix_a) -

array([[0.1, 0.8, 0. , 0.1],
       [0.9, 0.1, 0. , 0. ],
       [0. , 0.1, 0.1, 0.8],
       [0. , 0. , 0.9, 0.1]])

In [10]:
np.identity(transition_matrix_a.shape[0])

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [15]:
P = np.transpose(transition_matrix_a) - np.identity(transition_matrix_a.shape[0])
P

array([[-0.9,  0.8,  0. ,  0.1],
       [ 0.9, -0.9,  0. ,  0. ],
       [ 0. ,  0.1, -0.9,  0.8],
       [ 0. ,  0. ,  0.9, -0.9]])

In [14]:
A = np.append(P, np.ones((1, P.shape[0])), axis=0)
A

array([[-0.9,  0.8,  0. ,  0.1],
       [ 0.9, -0.9,  0. ,  0. ],
       [ 0. ,  0.1, -0.9,  0.8],
       [ 0. ,  0. ,  0.9, -0.9],
       [ 1. ,  1. ,  1. ,  1. ]])

In [17]:
np.zeros(P.shape[0])

array([0., 0., 0., 0.])

In [18]:
np.append(np.zeros(P.shape[0]), [1])

array([0., 0., 0., 0., 1.])

In [20]:
b = np.transpose(np.append(np.zeros(P.shape[0]), [1]))
b

array([0., 0., 0., 0., 1.])

In [21]:
np.transpose(A).dot(A)

array([[ 2.62, -0.53,  1.  ,  0.91],
       [-0.53,  2.46,  0.91,  1.16],
       [ 1.  ,  0.91,  2.62, -0.53],
       [ 0.91,  1.16, -0.53,  2.46]])